<a href="https://colab.research.google.com/github/giirrr/first_ryun_project/blob/main/model_test_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

import xgboost
import lightgbm as lgb
import catboost
import ngboost
import tensorflow as tf
import keras

from ngboost.learners import default_tree_learner
from ngboost.distns import Normal
from ngboost.scores import MLE

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mlp
import warnings
import sklearn
import joblib
import os
from random import randint

from tqdm import tqdm
warnings.filterwarnings('ignore')

"""data 불러오기"""
df_test = pd.read_csv('./train_data/test_data.csv')

df_test[df_test['AIR_PRESSURE'] < 900] = np.nan
df_test[df_test['AIR_PRESSURE'] > 1100 ] = np.nan

df_test[df_test['AIR_TEMPERATURE'] < -50] = np.nan
df_test[df_test['AIR_TEMPERATURE'] > 70 ] = np.nan

df_test[df_test['HUMIDITY'] < 0] = np.nan
df_test[df_test['HUMIDITY'] > 100 ] = np.nan

df_test[df_test['WIND_SPEED'] < 0] = np.nan
df_test[df_test['WIND_SPEED'] > 80 ] = np.nan

df_test = df_test.dropna()
df_test.info()

#MinMaxScaler 전처리
df_test['AIR_PRESSURE'] = (lambda ap : (ap-900) / 200)(df_test['AIR_PRESSURE'])
df_test['AIR_TEMPERATURE'] = (lambda at : (at+50) / 120)(df_test['AIR_TEMPERATURE'])
df_test['day_min'] = (lambda dm : dm / 1439)(df_test['day_min'])
df_test['HUMIDITY'] = (lambda h : h / 100)(df_test['HUMIDITY'])
df_test['WIND_SPEED'] = (lambda ws : ws / 80)(df_test['WIND_SPEED'])
#StandardScaler 전처리


feature_cols = ['AIR_PRESSURE', 'AIR_TEMPERATURE', 'day_min', 'HUMIDITY', 'WIND_SPEED']
label_cols = ['AIR_TEMPERATURE']

y_test = df_test[label_cols].values
X_test = df_test[feature_cols].values

X_test = X_test[:-1]
y_val = y_test[1:]

XG_X_test = X_test.copy()
LG_X_test = X_test.copy()
CB_X_test = X_test.copy()
NG_X_test = X_test.copy()
CG_X_test = X_test.copy()

"""model 불러오기"""
xgb_model = xgboost.XGBRegressor()
xgb_model.load_model("./learn_data/0epoch.json")
# xgb_model.load_model("./learn_data/0epoch11.json")
"optuna 적용"
lgb_model = lgb.LGBMRegressor
lgb_model = lgb.Booster(model_file='model1.txt')
# lgb_model = lgb.Booster(model_file='model2.txt')

cb_model = catboost.CatBoostRegressor()
cb_model.load_model("./learn_data/0epoch2.json")
#cb_model.load_model("./learn_data/0epoch22.json")

#ngb_model = ngboost.NGBRegressor(n_estimators=200,  natural_gradient=True, learning_rate = 0.01, Base = default_tree_learner, Dist=Normal, Score=MLE, verbose_eval =50)
ngb_model = joblib.load('./ngb_model.pkl')
#xgboost.plot_importance(xgb_model)

CG_model = keras.models.load_model('./learn_data/CGNN_tahn-Nadam-10dimention-LSTM/73-0.000050-0.00002.h5')

"""model 불러오기 끝"""


#non data [1074:1095]

con_drop_length = [20,40,60,150,300,450,600]
# con_drop_length = [20,40,60]

fig_sv_sw = True

mape_XG_space = np.zeros((len(con_drop_length), 1440))  ###################################
mape_LG_space = np.zeros((len(con_drop_length), 1440))  ###################################
mape_CB_space = np.zeros((len(con_drop_length), 1440))  ###################################
mape_li_space = np.zeros((len(con_drop_length), 1440))  ##################################
mape_ng_space = np.zeros((len(con_drop_length), 1440))  ##################################
mape_CG_space = np.zeros((len(con_drop_length), 1440))  ##################################
mape_flCG_space = np.zeros((len(con_drop_length), 1440))  ##################################

for cdl in range(len(con_drop_length)):
    tn = 1438 - con_drop_length[cdl]-2
    """현재 디렉토리에 있는 모든 파일 리스트를 가져와서 해당 파일 형식이 없으면 생성, 있으면 pass"""
    if 'time_delta_{}'.format(con_drop_length[cdl]) in os.listdir():
        pass
    else:
        os.mkdir('./time_delta_{}/'.format(con_drop_length[cdl]))
    # for try_num in tqdm(range(3, tn)):
    for try_num in tqdm(range(40)):
        drop_point = random.randint(3,tn)
        # drop_point = try_num   #continued drops start point 처음부터 끝까지(거의) 숫자 생성
        XG_X_test[drop_point: drop_point + con_drop_length[cdl] - 1, 1] = np.nan   # 생성된 정수 + 결측갯수 만큼 nun값 처리
        LG_X_test[drop_point: drop_point + con_drop_length[cdl] - 1, 1] = np.nan   # 생성된 정수 + 결측갯수 만큼 nun값 처리
        CB_X_test[drop_point: drop_point + con_drop_length[cdl] - 1, 1] = np.nan   # 생성된 정수 + 결측갯수 만큼 nun값 처리
        NG_X_test[drop_point: drop_point + con_drop_length[cdl] - 1, 1] = np.nan   # 생성된 정수 + 결측갯수 만큼 nun값 처리
        CG_X_test[drop_point: drop_point + con_drop_length[cdl] - 1, 1] = np.nan   # 생성된 정수 + 결측갯수 만큼 nun값 처리
        for i in range(1438):
            X_test_li = X_test.copy()
            # print(X_test_li[drop_point-1, 1])
            # print(X_test_li[drop_point + con_drop_length[cdl], 1])
            # print(np.linspace(X_test_li[drop_point-1, 1], X_test_li[drop_point + con_drop_length[cdl], 1], 22)[1:-1])
            X_test_li[drop_point : drop_point + con_drop_length[cdl], 1] = np.linspace(X_test_li[drop_point-1, 1], X_test_li[drop_point + con_drop_length[cdl], 1], con_drop_length[cdl]+2)[1:-1]
            if str(XG_X_test[i, 1]) == 'nan' or str(XG_X_test[i, 1]) == 'na':
                XG_X_test[i, 1] = xgb_model.predict(np.expand_dims(XG_X_test[i - 1, :], axis=0))

            if str(LG_X_test[i, 1]) == 'nan' or str(LG_X_test[i, 1]) == 'na':
                LG_X_test[i, 1] = lgb_model.predict(np.expand_dims(LG_X_test[i - 1, :], axis=0))

            if str(CB_X_test[i, 1]) == 'nan' or str(CB_X_test[i, 1]) == 'na':
                CB_X_test[i, 1] = cb_model.predict(np.expand_dims(CB_X_test[i - 1, :], axis=0))

            if str(NG_X_test[i, 1]) == 'nan' or str(NG_X_test[i, 1]) == 'na':
                NG_X_test[i, 1] = ngb_model.predict(np.expand_dims(NG_X_test[i - 1, :], axis=0))

            if str(CG_X_test[i, 1]) == 'nan' or str(CG_X_test[i, 1]) == 'na':
                CG_X_test[i, 1] = CG_model.predict(np.expand_dims(CG_X_test[i - con_drop_length[cdl], :], axis=0))
                #print(X_test[i-1, 1], XG_X_test[i-1, 1], LG_X_test[i-1, 1], CB_X_test[i-1, 1], NG_X_test[i-1, 1], CG_X_test[i-1, 1])

            """이미 그래프가 그려져있으면 pass 아니면 저장"""
        if fig_sv_sw == True:

            x_axis = np.arange(drop_point - 2, drop_point + con_drop_length[cdl] + 2)
            ### 원본,예측값 전체
            #fig, ax = plt.subplots(figsize=(10, 6))
            x = x_axis
            y = ((y_val[drop_point - 3: drop_point + con_drop_length[cdl] + 1] * 120) - 50)
            plt.plot(x, y, label='Actual')

            #y2 = ((XG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] * 120) - 50)
            #plt.plot(x, y2, label='XGBoosting_Predictionl')

            #y3 = ((LG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] * 120) - 50)
            #plt.plot(x, y3, label='LightGBM_Predictionl')

            #y4 = ((CB_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] * 120) - 50)
            #plt.plot(x, y4, label='CatBoosting_Predictionl')

            #y5 = ((NG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] * 120) - 50)
            #plt.plot(x, y5, label='NGBoosting_Predictionl')

            #y6 = (((CG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] ) * 120)-((CG_X_test[drop_point , 1]* 120)- (y_val[drop_point - 1 ]*120-50)))
            y6 = (((CG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] ) * 120)-50)
            plt.plot(x, y6, label='CGNN_Predictionl')

           # y7 = (((((CG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1]) * -1) + 1) * 120) - ((((CG_X_test[drop_point, 1] * -1) + 1) * 120) - (y_val[drop_point - 1] * 120 - 50)))
            #plt.plot(x, y7, label='VL_CGNN_Predictionl')

            y8 = ((X_test_li[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] * 120) - 50)
            plt.plot(x, y8, label='linear_Predictionl')
            plt.legend()
            plt.savefig('./time_delta_{0}/interpolate_graph_{1} to {2}'.format(con_drop_length[cdl], drop_point, drop_point + con_drop_length[cdl]))
            plt.close()
        else:
            #y = ((y_val[drop_point - 1: drop_point + con_drop_length[cdl] + 1] * 120) - 50)
            #y2 = ('XG : ', (XG_X_test[drop_point - 1: drop_point + con_drop_length[cdl] + 1, 1] * 120) - 50)
            #y3 = ('LG : ', (LG_X_test[drop_point - 1: drop_point + con_drop_length[cdl] + 1, 1] * 120) - 50)
            #y4 = ('CB : ', (CB_X_test[drop_point - 1: drop_point + con_drop_length[cdl] + 1, 1] * 120) - 50)
            #y5 = ('NG : ', (NG_X_test[drop_point - 1: drop_point + con_drop_length[cdl] + 1, 1] * 120) - 50)
            ##y6 = ('CG : ', (CG_X_test[drop_point - 1: drop_point + con_drop_length[cdl] + 1, 1] * 120) - 50)
            #y7 = ('li : ', (X_test_li[drop_point - 1: drop_point + con_drop_length[cdl] + 1, 1] * 120) - 50)
            #print(y)
            #print(y2)
            #print(y3)
            #print(y4)
            #print(y5)
            ##print(y6)
            #print(y7)
            pass

        """XGB 보간 mape값 계산"""
        XGP = XG_X_test[drop_point: drop_point + con_drop_length[cdl], 1]
        XGP = np.delete(XGP, np.where(XGP == 0))
        mape_XG = np.average(100 * np.abs((lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]) -
                           (lambda at: (at * 120) - 50)(XGP))
              / (lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]))

        """LGB 보간 mape값 계산"""
        LGP = LG_X_test[drop_point: drop_point + con_drop_length[cdl], 1]
        LGP = np.delete(LGP, np.where(LGP == 0))
        mape_LG = np.average(100 * np.abs((lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]) -
                                   (lambda at: (at * 120) - 50)(LGP))
                      / (lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]))

        """CB 보간 mape값 계산"""
        CGP = CB_X_test[drop_point: drop_point + con_drop_length[cdl], 1]
        CGP = np.delete(CGP, np.where(CGP == 0))
        mape_CB = np.average(
            100 * np.abs((lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]) -
                         (lambda at: (at * 120) - 50)(CGP))
            / (lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]))

        """NG 보간 mape값 계산"""
        NGP = NG_X_test[drop_point: drop_point + con_drop_length[cdl], 1]
        NGP = np.delete(NGP, np.where(NGP == 0))
        mape_NG = np.average(
            100 * np.abs((lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]) -
                         (lambda at: (at * 120) - 50)(NGP))
            / (lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]))

        """fl_CGNN 보간 mape값 계산"""
        fl_CGNP = ((((CG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] )*-1)+1)* 120)-((((CG_X_test[drop_point , 1]*-1)+1)* 120)- (y_val[drop_point - 1 ]*120-50))
        fl_CGNP = np.delete(fl_CGNP, np.where(fl_CGNP == 0))
        mape_flCG = np.average(
            100 * np.abs((lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]) -
                         (fl_CGNP))
            / (lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]))

        """CGNN 보간 mape값 계산"""
        CGNP = ((CG_X_test[drop_point - 2: drop_point + con_drop_length[cdl] + 2, 1] ) * 120)-((CG_X_test[drop_point , 1]* 120)- (y_val[drop_point - 1 ]*120-50))
        CGNP = np.delete(CGNP, np.where(CGNP == 0))
        mape_CG = np.average(
            100 * np.abs((lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]) -
                         (CGNP))
            / (lambda at: (at * 120) - 50)(y_val[drop_point: drop_point + con_drop_length[cdl]]))



        """선형 보간 mape값 계산"""
        mape_li = np.average(100 * np.abs((y_val[drop_point: drop_point + con_drop_length[cdl]]) -
                          X_test_li[drop_point: drop_point + con_drop_length[cdl], 1])
             / (y_val[drop_point: drop_point + con_drop_length[cdl]]))

        """만들어놓은 저장공간에 20,40,60 mape값 정리"""
        mape_XG_space[cdl][try_num] = mape_XG
        mape_LG_space[cdl][try_num] = mape_LG
        mape_CB_space[cdl][try_num] = mape_CB
        mape_ng_space[cdl][try_num] = mape_NG
        mape_CG_space[cdl][try_num] = mape_CG
        mape_flCG_space[cdl][try_num] = mape_flCG
        mape_li_space[cdl][try_num] = mape_li

        XG_X_test = X_test.copy()
        LG_X_test = X_test.copy()
        CB_X_test = X_test.copy()
        NG_X_test = X_test.copy()
        CG_X_test = X_test.copy()


mape_mXG = []
mape_mLG = []
mape_mCB = []
mape_mNG = []
mape_mCG = []
mape_mflCG = []
mape_mli = []

time_stamp = []
for cd in con_drop_length:
    time_stamp.append('time delta %d :'%cd)

# np.savetxt('mape_XG_space.csv', mape_XG_space, delimiter=',')
# np.savetxt('mape_LG_space.csv', mape_LG_space, delimiter=',')
# np.savetxt('mape_CB_space.csv', mape_CB_space, delimiter=',')
# np.savetxt('mape_ng_space.csv', mape_ng_space, delimiter=',')
# np.savetxt('mape_CG_space.csv', mape_CG_space, delimiter=',')
# np.savetxt('mape_flCG_space.csv', mape_flCG_space, delimiter=',')
# np.savetxt('mape_li_space.csv', mape_li_space, delimiter=',')

"""XGB,Linear 보간법 별 반복횟수 만큼 생성된 모델의 평균,분산값"""
for i in range(len(con_drop_length)):
    mape_mXG.append('XGBoost interpolate {0}\u03bc={1: .5f} \u03c3={2: .5f} '.format(time_stamp[i], np.average(mape_XG_space[i]), np.std(mape_XG_space[i])))
    mape_mLG.append('LGBoost interpolate {0}\u03bc={1: .5f} \u03c3={2: .5f} '.format(time_stamp[i], np.average(mape_LG_space[i]), np.std(mape_LG_space[i])))
    mape_mCB.append('CatGBoost interpolate {0}\u03bc={1: .5f} \u03c3={2: .5f} '.format(time_stamp[i], np.average(mape_CB_space[i]), np.std(mape_CB_space[i])))
    mape_mNG.append('NGBoost interpolate {0}\u03bc={1: .5f} \u03c3={2: .5f} '.format(time_stamp[i], np.average(mape_ng_space[i]), np.std(mape_ng_space[i])))
    mape_mCG.append('CGNN interpolate {0}\u03bc={1: .5f} \u03c3={2: .5f} '.format(time_stamp[i], np.average(mape_CG_space[i]), np.std(mape_CG_space[i])))
    mape_mflCG.append('vertical_flip_CGNN interpolate {0}\u03bc={1: .5f} \u03c3={2: .5f} '.format(time_stamp[i], np.average(mape_flCG_space[i]), np.std(mape_flCG_space[i])))
    mape_mli.append('linear interpolate {0}\u03bc={1: .5f} \u03c3={2: .5f}'.format(time_stamp[i], np.average(mape_li_space[i]), np.std(mape_li_space[i])))

"""위의 값을 가지고 정규분포를 따르는지 히스토그램을 그려 확인(신뢰성)"""
for j in range(len(con_drop_length)):
    print('{}'.format(mape_mXG[j]))
    print('{}'.format(mape_mLG[j]))
    print('{}'.format(mape_mCB[j]))
    print('{}'.format(mape_mNG[j]))
    print('{}'.format(mape_mCG[j]))
    print('{}'.format(mape_mflCG[j]))
    print('{}\n'.format(mape_mli[j]))
#    fig, ax = plt.subplots(figsize=(10, 6))
#    plt.title('XGBoost_error(percent)_distribution')
#    plt.hist(mape_XG_space[j], bins=100, histtype='stepfilled')
#    plt.xticks(np.arange(0, np.max(mape_XG_space[j])+1, 0.5))
#    plt.xlim(0, 5)
#    plt.show()
#
#    plt.title('LGBoost_error(percent)_distribution')
#    plt.hist(mape_LG_space[j], bins=100, histtype='stepfilled')
#    plt.xticks(np.arange(0, np.max(mape_LG_space[j])+1, 0.5))
#    plt.xlim(0, 5)
#    plt.show()
#
#    plt.title('CatBoost_error(percent)_distribution')
#    plt.hist(mape_CB_space[j], bins=100, histtype='stepfilled')
#    plt.xticks(np.arange(0, np.max(mape_CB_space[j]) + 1, 0.5))
#    plt.xlim(0, 5)
#    plt.show()
#
#    plt.title('NGBoost_error(percent)_distribution')
#    plt.hist(mape_ng_space[j], bins=100, histtype='stepfilled')
#    plt.xticks(np.arange(0, np.max(mape_ng_space[j])+1, 0.5))
#    plt.xlim(0, 5)
#    plt.show()
#
#    #plt.title('CGNN_error(percent)_distribution')
#    #plt.hist(mape_CG_space[j], bins=100, histtype='stepfilled')
#    #plt.xticks(np.arange(0, np.max(mape_CG_space[j])+1, 0.5))
#    #plt.xlim(0, 5)
#    #plt.show()
#
#    fig, ax = plt.subplots(figsize=(10, 6))
#    plt.title('linear_error(percent)_distribution')
#    plt.hist(mape_li_space[j], bins=100, histtype='stepfilled')
#    plt.xticks(np.arange(0, np.max(mape_li_space[j]) + 1, 0.1))
#    plt.xlim(0, 1)
#    plt.show()